In [3]:
import pandas as pd
import numpy
import os
import plotly.express as px

df = pd.read_csv('./data/Speed+Dating+Data.csv',low_memory=False,encoding='cp1252')
display(df.head())
display(df.shape)
display(df.columns)

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


(8378, 195)

Index(['iid', 'id', 'gender', 'idg', 'condtn', 'wave', 'round', 'position',
       'positin1', 'order',
       ...
       'attr3_3', 'sinc3_3', 'intel3_3', 'fun3_3', 'amb3_3', 'attr5_3',
       'sinc5_3', 'intel5_3', 'fun5_3', 'amb5_3'],
      dtype='object', length=195)

In [19]:
fig = px.bar(df['gender'].value_counts().reset_index(),x='gender',y='count',title='Distribution gender')
fig.show()

In [21]:
print(f'Nombre de valeur vide : \n{df.isna().sum()}')
print('------')
print(df['career'].value_counts().head(10))


Nombre de valeur vide : 
iid            0
id             1
gender         0
idg            0
condtn         0
            ... 
attr5_3     6362
sinc5_3     6362
intel5_3    6362
fun5_3      6362
amb5_3      6362
Length: 195, dtype: int64
------
career
Finance          202
professor        199
Lawyer           154
Professor        148
Consulting       147
Social Worker    136
Law              121
Business         103
Academic         101
Scientist         91
Name: count, dtype: int64
